In [ ]:
import cv2
import numpy as np
import supervision as sv
from ultralytics import SAM

from tqdm import tqdm


def click_event(event, x, y, flags, param):
    global point
    if event == cv2.EVENT_LBUTTONDOWN:
        point = (x, y)
        print(f"Point selected: {point}")


# Load a model
model = SAM("sam2_t.pt")

video_path = fr"videos/IMG_0036.mp4"
target_path = video_path.replace(".mp4", "_segmented.mp4")
frame_generator = sv.get_video_frames_generator(source_path=video_path)
video_info = sv.VideoInfo.from_video_path(video_path=video_path)

mask_annotator = sv.PolygonAnnotator()
box_annotator = sv.BoxAnnotator()
labeler = sv.LabelAnnotator()

# Initialize global variables to store the point coordinates
point = None


cv2.namedWindow('Predictions')
cv2.setMouseCallback('Predictions', click_event)

# Open the video sink
with sv.VideoSink(target_path=target_path, video_info=video_info) as sink:
    for frame in tqdm(frame_generator, total=video_info.total_frames):

        if point is not None:
            # Use the selected point for processing
            results = model(frame, points=[point], labels=[1])[0]
            masks = results.masks.data.cpu().numpy()
            xyxy = sv.mask_to_xyxy(masks)
            cls = np.array([0] * len(xyxy))
            detections = sv.Detections(xyxy, masks, class_id=cls)

            # Annotate the frame with boxes, masks, and labels
            frame = box_annotator.annotate(scene=frame, detections=detections)
            frame = mask_annotator.annotate(scene=frame, detections=detections)

        # Display frame and wait for a click event
        cv2.imshow('Predictions', frame)
        key = cv2.waitKey(1)

        # Break the loop if the 'q' key is pressed
        if key & 0xFF == ord('q'):
            break

cv2.destroyAllWindows()

## Run Inference with TensorRT 使用 TensorRT 运行推理

In [ ]:
# Enable MAX Power Mode and Jetson Clocks
!sudo nvpmodel -m 0
!sudo jetson_clocks

In [ ]:
# Monitor System Utilization
!sudo apt update
!sudo pip install jetson-stats
!sudo reboot
!jtop

In [4]:
%pip install imutils sort

ERROR: Could not find a version that satisfies the requirement sort (from versions: none)
ERROR: No matching distribution found for sort
Note: you may need to restart the kernel to use updated packages.


In [3]:
import cv2
import torch
import imutils
from ultralytics import YOLO
# Kalman filtering 卡尔曼滤波器
from sort import *


device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")
torch.cuda.set_device(0)
torch.set_default_tensor_type(torch.cuda.FloatTensor)
model = YOLO('best_prep.pt').to(device)

video_path = "videos/IMG_0036.MOV"
cap = cv2.VideoCapture(video_path)
# https://medium.com/@mosesdaudu001/object-detection-tracking-with-yolov8-and-sort-algorithm-363be8bc0806
sort_tracker = Sort(max_age=20, min_hits=2, iou_threshold=0.3)

isFirstFrame = True

t1 = time.time()
fc = 0
while True:
	ret, frame = cap.read()
	if not ret:
		break
	fc = fc + 1

	if(isFirstFrame):
		isFirstFrame = False

	results = model(frame)

	dets_to_sort = np.empty((0, 6))
	for result in results:
		for obj in result.boxes:
			bbox = obj.xyxy[0].cpu().numpy().astype(int)
			x1, y1, x2, y2 = bbox

			conf = obj.conf.item()
			class_id = int(obj.cls.item())
			dets_to_sort = np.vstack((dets_to_sort, np.array([x1, y1, x2, y2, conf, class_id])))
			# cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

	# Sort tracker 对检测结果进行跟踪, 更新卡尔曼滤波器的状态
	tracked_dets = sort_tracker.update(dets_to_sort)
	for det in tracked_dets:
		x1, y1, x2, y2 = [int(i) for i in det[:4]]
		track_id = int(det[8]) if det[8] is not None else 0
		class_id = int(det[4])
		cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 4)
		cv2.putText(frame, f"{track_id}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 3)

	# 预测后的两帧的上下相对位置关系比较
	frame = imutils.resize(frame, width=800)
	# cv2.imshow('Frame', frame)
	key = cv2.waitKey(1)
	if key == ord('q'):
		break
	if key == ord('p'):
		cv2.waitKey(-1)

cap.release()
cv2.destroyAllWindows()
t2 = time.time()
ft = t2 - t1
print(fc)
print('Execution time {}'.format(ft))
print('FPS: {}'.format(fc / ft))

ModuleNotFoundError: No module named 'sort'

## 优化YOLOv8预处理和后处理时间

In [ ]:
##1. Batch Processing 批处理：如果要处理多个图像，将它们一起批处理可以显着减少每个图像的开销。您可以在predict调用中调整batch参数：

results = model.predict(["cat.jpg", "dog.jpg"], batch=2, conf=0.7)

##2. Reduce Image Size 减小图像大小

##3. 使用混合精度 
results = model.predict("cat.jpg", conf=0.7, half=True)

##4. cProfile或line_profiler分析工具来识别代码中的耗时瓶颈

##4. Parallel Processing处理多个图像